# Missing Values Exercise

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# readin the data

X_full=pd.read_csv('train.csv',index_col='Id')
X_test_full=pd.read_csv('test.csv', index_col='Id')

In [3]:
# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
 # removing rows with missing target
y = X_full.SalePrice #Setting predicting variable
X_full.drop(['SalePrice'], axis=1, inplace=True) # removing prediting column from x data

In [4]:
# we are using only numerical data which means that we will exclude object types data
X = X_full.select_dtypes(exclude=['object'])
X_test = X_test_full.select_dtypes(exclude=['object'])

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [5]:
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,...,774,0,108,0,0,260,0,0,7,2007
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,...,308,0,0,0,0,0,0,0,8,2009
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,...,432,0,0,44,0,0,0,0,8,2009
818,20,NaN,13265,8,5,2002,2002,148.0,1218,0,...,857,150,59,0,0,0,0,0,7,2008
303,20,118.0,13704,7,5,2001,2002,150.0,0,0,...,843,468,81,0,0,0,0,0,1,2006


# 1. Step : Preliminary investigation

In [6]:
X_train.shape

(1168, 36)

We have 1168 rows and 36 columns

In [7]:
X_train.isnull().sum()

MSSubClass         0
LotFrontage      212
LotArea            0
OverallQual        0
OverallCond        0
YearBuilt          0
YearRemodAdd       0
MasVnrArea         6
BsmtFinSF1         0
BsmtFinSF2         0
BsmtUnfSF          0
TotalBsmtSF        0
1stFlrSF           0
2ndFlrSF           0
LowQualFinSF       0
GrLivArea          0
BsmtFullBath       0
BsmtHalfBath       0
FullBath           0
HalfBath           0
BedroomAbvGr       0
KitchenAbvGr       0
TotRmsAbvGrd       0
Fireplaces         0
GarageYrBlt       58
GarageCars         0
GarageArea         0
WoodDeckSF         0
OpenPorchSF        0
EnclosedPorch      0
3SsnPorch          0
ScreenPorch        0
PoolArea           0
MiscVal            0
MoSold             0
YrSold             0
dtype: int64

In [8]:
# if we remove all missing values then we keep percentage of data:

(1-270/1168)*100

76.88356164383562

In [9]:
# again defining function for model score

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

# 2. STEP:  Drop columns with missing values

In [11]:
cols_with_missing_values=[col for col in X_train.columns if X_train[col].isnull().any()]

In [12]:
reduced_X_train=X_train.drop(cols_with_missing_values, axis=1)
reduced_X_valid=X_valid.drop(cols_with_missing_values, axis=1)

In [13]:
print("MAE from Approach 1 (drop columns with missing values):")
print(score_dataset(reduced_X_train,reduced_X_valid,y_train,y_valid))

MAE from Approach 1 (drop columns with missing values):
17837.82570776256


# 3. Step : Imputation

In [14]:
from sklearn.impute import SimpleImputer

In [15]:
my_imputer=SimpleImputer()
imputed_X_train=pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid=pd.DataFrame(my_imputer.transform(X_valid))

#imputation removed column names; puttinh them back

imputed_X_train.columns=X_train.columns
imputed_X_valid.columns=X_valid.columns

In [16]:
print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE from Approach 2 (Imputation):
18062.894611872147


In [17]:
# as final data we will take reduced data

In [18]:
model=RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(reduced_X_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [19]:
pred=model.predict(reduced_X_valid)

print("MAE (Your appraoch):")
print(mean_absolute_error(y_valid, pred))

MAE (Your appraoch):
17837.82570776256


In [20]:
# Imputation - imputation based on median - middle value
final_imputer = SimpleImputer(strategy='median')
final_X_train = pd.DataFrame(final_imputer.fit_transform(X_train))
final_X_valid = pd.DataFrame(final_imputer.transform(X_valid))

# Imputation removed column names; put them back
final_X_train.columns = X_train.columns
final_X_valid.columns = X_valid.columns

In [21]:
# Define and fit model
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(final_X_train, y_train)

# Get validation predictions and MAE
preds_valid = model.predict(final_X_valid)
print("MAE (Your appraoch):")
print(mean_absolute_error(y_valid, preds_valid))

MAE (Your appraoch):
17791.59899543379


In [22]:
#best result!!! for final data we are using imputer method

In [23]:
# Preprocess test data - transforming test data 
final_X_test = pd.DataFrame(final_imputer.transform(X_test))

# Get test predictions
preds_test = model.predict(final_X_test)

In [24]:
# Save test predictions to file
output = pd.DataFrame({'Id': final_X_test.index,
                       'SalePrice': preds_test})
output.to_csv('Prediction.csv', index=False)